# Toy Radix Lagrangean decomposition validation

In [1]:
REG_WEIGHT   = 1e-4
MAX_NONZERO  = 48*2

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from cobra.io import load_json_model
from six import iteritems
import numpy as np
import cobra

In [4]:
ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [6]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if ijomc.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

# Make radix problem

In [7]:
from dynamicme.estimate import RadixEstimator

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f3a07b17368>)


In [8]:
df_Y = df_meas.rename(columns={'growth_rate_1_h':'output', 'substrate':'cond'})
df_Y.loc[:,'output_id'] = 'BIOMASS_Ecoli_core_w_GAM'
df_X = df_conds

In [9]:
est = RadixEstimator()
est.fit(ijomc, df_X, df_Y, reg_weight = REG_WEIGHT, max_nonzero_binaries=MAX_NONZERO, optimize=False)
est.milp.Params.ScaleFlag = 0
est.milp.Params.OutputFlag = 1
# est.milp.Params.Presolve = 0
# est.milp.Params.FeasibilityTol = 1e-8
# est.milp.Params.OptimalityTol = 1e-8
est.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter ScaleFlag to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 2837 rows, 1440 columns and 8462 nonzeros
Variable types: 1296 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+03]
  Objective range  [1e-04, 8e-01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e-03, 1e+03]
Presolve removed 735 rows and 260 columns
Presolve time: 0.02s
Presolved: 2102 rows, 1180 columns, 6702 nonzeros
Variable types: 1036 continuous, 144 integer (144 binary)
Presolve removed 2 rows and 4 columns

Root relaxation: objective 3.187023e-03, 1405 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00319    0   98          -    0.

<Solution 0.00 at 0x7f3a582eb5d0>

## Validate the fit

In [10]:
base_model = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')

In [11]:
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
df_pred = est.predict(df_X, base_model, meas_id=mu_id)

In [12]:
df_Y2 = df_Y[[c for c in df_Y.columns if c not in ['order','ex_rxn']]]
df_val = pd.merge(df_Y2.rename(columns={'output':'meas'}), df_pred.rename(columns={'rxn':'output_id','x':'pred'}), on=['cond','output_id'])
df_val.loc[:,'error'] = (df_val.pred - df_val.meas)
df_val.loc[:,'perror'] = df_val.error / df_val.meas * 100
df_val

,cond,meas,output_id,pred,error,perror
0,glucose,0.740,BIOMASS_Ecoli_core_w_GAM,0.740174,0.000174,0.023479
1,acetate,0.256,BIOMASS_Ecoli_core_w_GAM,0.256324,0.000324,0.126470


In [13]:
a0 = 1./65/3600
kfit_dict0 = {k:a0 for k in est.kfit_dict.keys()}
df_ref = est.predict(df_X, base_model, meas_id=mu_id, kfit_dict=kfit_dict0)
df_ref.rename(columns={'rxn':'output_id','x':'output'}, inplace=True)

In [14]:
df_val0 = pd.merge(df_Y2.rename(columns={'output':'meas'}), df_ref.rename(columns={'output':'pred0'}), on=['cond','output_id'])
df_val0.loc[:,'error0'] = (df_val0.pred0 - df_val0.meas)
df_val0.loc[:,'perror0'] = df_val0.error0 / df_val0.meas * 100

In [15]:
df_comp = pd.merge(df_val, df_val0, on=['cond','output_id','meas'])
df_comp.loc[:,'error_change'] = (df_comp.error.abs()-df_comp.error0.abs()) / df_comp.error0.abs() * 100
df_comp

,cond,meas,output_id,pred,error,perror,pred0,error0,perror0,error_change
0,glucose,0.740,BIOMASS_Ecoli_core_w_GAM,0.740174,0.000174,0.023479,0.873922,0.133922,18.097501,-99.870263
1,acetate,0.256,BIOMASS_Ecoli_core_w_GAM,0.256324,0.000324,0.126470,0.389313,0.133313,52.075420,-99.757140


In [16]:
print('Total abs error: %g%%' % df_comp.perror.abs().sum())
print('Total error change: %g%%' % df_comp.error_change.sum())
changed = [abs(est.kfit_dict[k]-kfit_dict0[k])>1e-9 for k in est.kfit_dict.keys()]
print('Number of keffs changed: %d/%d (%g%%)' % (sum(changed),len(est.kfit_dict), 100*sum(changed)/len(est.kfit_dict) ))

Total abs error: 0.149949%
Total error change: -199.627%
Number of keffs changed: 31/48 (64%)


# Now, use optimal Lagrange multipliers and verify same solution with relaxation

In [17]:
param_dict = {}

In [26]:
csrcs = df_conds.cond.unique()
for csrc in csrcs:
    ijofit = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
    crowding = ijofit.metabolites.get_by_id('crowding')    
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']
    
    ### Formulate problem for one condition with the non-anticipativity constraint relaxation            
    df_xi = df_X[df_X.cond==csrc]
    df_yi = df_Y[df_Y.cond==csrc]
    esti = RadixEstimator()
    esti.fit(ijomc, df_xi, df_yi, reg_weight = REG_WEIGHT, max_nonzero_binaries=MAX_NONZERO, optimize=False)
    esti.milp.Params.ScaleFlag = 0
    esti.milp.Params.OutputFlag = 0
    # Get optimal Lagrange multipliers from actual solution
        
    # Add the non-anticipativity
    
    
    esti.optimize()
    esti.get_params()
    param_dict[csrc] = esti.kfit_dict

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter ScaleFlag to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter ScaleFlag to 0
   Prev: -1  Min: -1  Max: 3  Default: -1


In [ ]:
    # ijofit.optimize()
    
#     mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
#     mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
#     # Get unfit
#     for rxn in ijofit.metabolites.crowding.reactions:
#         rxn._metabolites[crowding] = a0
#     ijofit.optimize()
#     mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
#     err0= 100*(mu_unfiti-mu_measi)/mu_measi
#     err = 100*(mu_fiti - mu_measi)/mu_measi
#     derr= 100*(abs(err)-abs(err0))/abs(err0)
#     print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
#     for i,row in df_condi.iterrows():
#         rid = row['rxn']
#         rxn = ijofit.reactions.get_by_id(rid)        
#         print('\t%s uptake=%g' % (rxn.id, rxn.x))

In [30]:
ub=0.00623
lb=-0.55015
(ub-lb)/(1e-10+ub) * 100

8930.65796258976